# packages and code

In [52]:
from tps import ThinPlateSpline
from tps_scripts import *

# paths

In [53]:
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

def dirUp(path, levels=1):
    """
    Move up the directory structure by a specified number of levels.
    """
    for _ in range(levels):
        path = os.path.dirname(path)
    return path

Current working directory: c:\Git\1_current_projects\powerlifing_model\models\athlete_03\001bis\P01\model_update


In [54]:
# ensure correct individual name
root = dirUp(cwd, 3)
print(f"Root directory: {root}")
tmp = cwd.split('\\')[len(root.split('\\')):]
ind = 'P' + tmp[0][1:]
ind

Root directory: c:\Git\1_current_projects\powerlifing_model\models\athlete_03


'P01bis'

In [55]:
# paths
mri_dir = os.path.abspath('../../MRI/results')
path_to_json = os.path.join(mri_dir, 'orientation.mrk.json')
template_dir = os.path.abspath('../../MRI/template/template_3D.json')

xml_path = os.path.abspath(os.path.join(root,'../Athlete_03_markers_lowerbody_katya.xml'))
osim_path = os.path.abspath(os.path.join(root,'../Athlete_03_lowerBody_final.osim'))

working_dir = os.getcwd()
output_path = os.path.join(working_dir, 'tps_warping_results')

vtp_path = os.path.join(working_dir, 'Geometry')

In [56]:
# check if paths exist
if not os.path.exists(mri_dir):
    print(f"Directory {mri_dir} does not exist.")

if not os.path.exists(path_to_json):
    print(f"File {path_to_json} does not exist.")
    
if not os.path.exists(xml_path):
    print(f"File {xml_path} does not exist.")
    
if not os.path.exists(osim_path):
    print(f"File {osim_path} does not exist.")
    
if not os.path.exists(working_dir):
    print(f"Directory {working_dir} does not exist.")
    
if not os.path.exists(output_path):
    print(f"Directory {output_path} does not exist.")
    
if not os.path.exists(vtp_path):
    print(f"Directory {vtp_path} does not exist.")

In [57]:
# test output path
if working_dir.split('\\')[-1] != 'model_update':
    output_root = 'model_update'
else: output_root = '.'

# set output path
output_path = os.path.join(output_root, 'tps_warping_results')
if not os.path.exists(output_path):
    print(output_path)
    os.makedirs(output_path)

# set path for controls
control_path = os.path.join(output_root, 'tps_warping_results', 'control')
if not os.path.exists(control_path):
    print(control_path)
    os.makedirs(control_path)

# load osim

In [58]:
# OSIM
osim_bone_markers = OsimBoneMarkers(xml_path)
osim_bone_markers_df = osim_bone_markers.data_frame()

In [59]:
osim_muscles_and_wrapping = OsimMusclePathsAndWrapping(osim_path)

osim_muslce_paths_df = osim_muscles_and_wrapping.df_muscles

wrapping_surfaces_df = osim_muscles_and_wrapping.wrp_df

# load mri and match to osim

In [60]:
# MRI
mri_bone_markers = MRIBoneMarkers(path_to_json)
mri_bone_markers_df = mri_bone_markers.json_to_df()

# collections by bodies

In [61]:
osim_mri_bone_by_bodies = OsimMriBoneByBodies(osim_bone_markers_df, mri_bone_markers_df)
osim_skin_markers_dict = osim_mri_bone_by_bodies.split_osim_skin_by_bodies()

osim_dict = osim_mri_bone_by_bodies.split_osim_by_bodies()
mri_dict = osim_mri_bone_by_bodies.split_mri_by_osim_bodies()

osim_muscls_by_bodies = OsimMusclesByBodies(osim_muslce_paths_df)
osim_muscls_dict = osim_muscls_by_bodies.extract_all()

osim_wraps_by_bodies = OsimWrapsByBodies(wrapping_surfaces_df)
osim_wraps_dict = osim_wraps_by_bodies.wraps_to_points_by_bodies()

# pelvis tps

In [62]:
# exclude markers to control for the quality of the spline
exclude = ['sacroiliac_r', 'sacroiliac_l']#,'5th_sacr_v'

# create the spline
pelvis_tps = OneBodyTPS('pelvis', osim_dict['pelvis'], mri_dict['pelvis'], 
                        osim_skin = osim_skin_markers_dict['pelvis'],
                        osim_muscle = osim_muscls_dict['pelvis'], 
                        exclude_bone_markers = exclude)

# transform all objects: bone markers are transformed and should fit the chosen subset save for the exclusions
osim_bone = osim_dict['pelvis']
pelvis_transformed_bone = pelvis_tps.transformed_bone
pelvis_transformed_muscle = pelvis_tps.transformed_muscle
pelvis_transformed_wraps = pelvis_tps.apply_tps_to_wraps(osim_wraps_dict['pelvis'])
pelvis_transformed_skin = pelvis_tps.transformed_skin

l_pelvis = ImportVTP(vtp_path, 'l_pelvis.vtp')
r_pelvis = ImportVTP(vtp_path, 'r_pelvis.vtp')
sacrum = ImportVTP(vtp_path, 'sacrum.vtp')

l_pelvis_transformed = pv.PolyData(pelvis_tps.apply_spline_to_surface(l_pelvis.mesh))
r_pelvis_transformed = pv.PolyData(pelvis_tps.apply_spline_to_surface(r_pelvis.mesh))
sacrum_transformed = pv.PolyData(pelvis_tps.apply_spline_to_surface(sacrum.mesh))

# plot resutls
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_pelvis.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(r_pelvis.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(sacrum.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(l_pelvis_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_pelvis_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(sacrum_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pelvis_transformed_bone),  color='blue', point_size=10.0)
if len(exclude) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['pelvis'].loc[exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f0ff91fe90_8&reconnect=auto" class="pyvis…

In [63]:
# #record results into the control folder
pelvis_transformed_bone_df = pd.DataFrame(pelvis_transformed_bone, index=osim_bone.index, columns=osim_bone.columns)
pelvis_transformed_skin_df = pd.DataFrame(pelvis_transformed_skin, index=osim_skin_markers_dict['pelvis'].index, columns=osim_skin_markers_dict['pelvis'].columns)

pelvis_transformed_muscle_df = pd.DataFrame(pelvis_transformed_muscle, index=osim_muscls_dict['pelvis'].index, columns=osim_muscls_dict['pelvis'].columns)
pelvis_transformed_wraps_translation_df = pd.DataFrame(pelvis_transformed_wraps[2], index=osim_wraps_dict['pelvis']['name'], columns=['r', 'a', 's'])

pelvis_transformed_bone_df.to_csv(os.path.join(control_path, 'pelvis_transformed_bone.csv'))
pelvis_transformed_skin_df.to_csv(os.path.join(control_path, 'pelvis_transformed_skin.csv'))

pelvis_transformed_muscle_df.to_csv(os.path.join(control_path, 'pelvis_transformed_muscle.csv'))
pelvis_transformed_wraps_translation_df.to_csv(os.path.join(control_path, 'pelvis_transformed_wraps_translation.csv'))

l_pelvis_transformed.save(os.path.join(control_path, f'l_pelvis_transformed.stl'))
r_pelvis_transformed.save(os.path.join(control_path, f'r_pelvis_transformed.stl'))
sacrum_transformed.save(os.path.join(control_path, f'sacrum_transformed.stl'))

In [64]:
# rotate results to child frame
bone_markers = list(mri_dict['pelvis'].index)
pelvis_axes = GetPelvisAxes(pelvis_transformed_bone, bone_markers)

pelvis_bone_in_child = pelvis_axes.bone_transformed
pelvis_skin_in_child = pelvis_axes.apply_to_non_bone(pelvis_transformed_skin)
pelvis_muscles_in_child = pelvis_axes.apply_to_non_bone(pelvis_transformed_muscle)
pelvis_wrap_translations_in_child = pelvis_axes.apply_to_non_bone(pelvis_transformed_wraps[2])

l_pelvis_surface_in_child = pv.PolyData(pelvis_axes.apply_to_non_bone(l_pelvis_transformed.points), l_pelvis_transformed.faces)
r_pelvis_surface_in_child = pv.PolyData(pelvis_axes.apply_to_non_bone(r_pelvis_transformed.points), r_pelvis_transformed.faces)
sacrum_surface_in_child = pv.PolyData(pelvis_axes.apply_to_non_bone(sacrum_transformed.points), sacrum_transformed.faces)

the determinant is less than zero, recalculate r


In [65]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_pelvis.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(r_pelvis.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(sacrum.mesh), color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(osim_bone.to_numpy()),  color='blue', point_size=10.0) 
actor = plotter.add_mesh(pv.PolyData(osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['pelvis'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['pelvis']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(l_pelvis_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_pelvis_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(sacrum_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pelvis_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(pelvis_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(pelvis_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f12c4749d0_9&reconnect=auto" class="pyvis…

# femur_r  tps

In [66]:
# exclude markers to control for the quality of the spline
to_exclude_femur_r = [] #'knee_r_center_in_femur_r','gr_troch_as_r', 'femur_r_poster_diaph_25', 'femur_r_anter_diaph_25', 

# calculate the spline
femur_r_tps = OneBodyTPS('femur_r', osim_dict['femur_r'], 
                         mri_dict['femur_r'], 
                        osim_muscle = osim_muscls_dict['femur_r'], 
                         osim_skin = osim_skin_markers_dict['femur_r'],
                         exclude_bone_markers = to_exclude_femur_r)

# transform all objects
femur_r_osim_bone = osim_dict['femur_r']
femur_r_transformed_bone = femur_r_tps.transformed_bone
femur_r_transformed_skin = femur_r_tps.transformed_skin
femur_r_transformed_muscle = femur_r_tps.transformed_muscle
femur_r_transformed_wraps = femur_r_tps.apply_tps_to_wraps(osim_wraps_dict['femur_r'])

r_femur = ImportVTP(vtp_path, 'r_femur.stl')
femur_r_transformed = femur_r_tps.apply_spline_to_surface(r_femur.mesh)

# plot results
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_transformed_bone),  color='blue', point_size=10.0)
if len(to_exclude_femur_r) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['femur_r'].loc[to_exclude_femur_r].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')


Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f12c55e1d0_10&reconnect=auto" class="pyvi…

In [67]:
# # record results into the control folder
femur_r_transformed_bone_df = pd.DataFrame(femur_r_transformed_bone, index=femur_r_osim_bone.index, columns=femur_r_osim_bone.columns)
femur_r_transformed_skin_df = pd.DataFrame(femur_r_transformed_skin, index=osim_skin_markers_dict['femur_r'].index, columns=osim_skin_markers_dict['femur_r'].columns)

femur_r_transformed_muscle_df = pd.DataFrame(femur_r_transformed_muscle, index=osim_muscls_dict['femur_r'].index, columns=osim_muscls_dict['femur_r'].columns)
femur_r_transformed_wraps_translation_df = pd.DataFrame(femur_r_transformed_wraps[2], index=osim_wraps_dict['femur_r']['name'], columns=['r', 'a', 's'])

femur_r_transformed_bone_df.to_csv(os.path.join(control_path, 'femur_r_transformed_bone.csv'))
femur_r_transformed_skin_df.to_csv(os.path.join(control_path, 'femur_r_transformed_skin.csv'))

femur_r_transformed_muscle_df.to_csv(os.path.join(control_path, 'femur_r_transformed_muscle.csv'))
femur_r_transformed_wraps_translation_df.to_csv(os.path.join(control_path, 'femur_r_transformed_wraps_translation.csv'))

femur_r_transformed.save(os.path.join(control_path, f'femur_r_transformed.stl'))

In [68]:
# create rotations to child
bone_markers = list(mri_dict['femur_r'].index)
femur_r_axes = GetFemurAxes(femur_r_transformed_bone,
                            bone_markers, 
                            femur_skin_numpy = femur_r_transformed_skin,
                            femur_muscles_numpy = femur_r_transformed_muscle,
                            femur_wraps_numpy = femur_r_transformed_wraps[2],
                            femur_surface_numpy = femur_r_transformed.points)
# rotate to child
femur_r_bone_in_child = femur_r_axes.bone_transformed
femur_r_skin_in_child = femur_r_axes.femur_skin
femur_r_muscles_in_child = femur_r_axes.femur_muscles
femur_r_wrap_translations_in_child = femur_r_axes.femur_wraps
femur_r_surface_in_child = pv.PolyData(femur_r_axes.femur_surface, femur_r_transformed.faces)

In [69]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['femur_r'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['femur_r']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_r_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(femur_r_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(femur_r_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f14aed2750_11&reconnect=auto" class="pyvi…

# patella_r tps

In [70]:
# exclude markers
patella_r_exclude =  None

# calculate spline
patella_r_tps = OneBodyTPS('patella_r', osim_dict['patella_r'], 
                         mri_dict['patella_r'], 
                         osim_muscle = osim_muscls_dict['patella_r'], 
                         osim_skin = None, 
                         exclude_bone_markers = patella_r_exclude)

# transform objects
patella_r_osim_bone = osim_dict['patella_r']
patella_r_transformed_bone = patella_r_tps.transformed_bone
patella_r_transformed_muscle = patella_r_tps.transformed_muscle

r_patella = ImportVTP(vtp_path, 'r_patella.vtp')

patella_r_transformed = patella_r_tps.apply_spline_to_surface(r_patella.mesh)


# plot results
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_transformed_bone),  color='blue', point_size=10.0)
#if len(patella_r_exclude) >= 1:
#actor = plotter.add_mesh(pv.PolyData(osim_mri_bone_by_bodies.patella_r_mri.loc[patella_r_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f14afc0990_12&reconnect=auto" class="pyvi…

In [71]:
# # record results into the control folder
patella_r_transformed_bone_df = pd.DataFrame(patella_r_transformed_bone, index=patella_r_osim_bone.index, columns=patella_r_osim_bone.columns)
patella_r_transformed_muscle_df = pd.DataFrame(patella_r_transformed_muscle, index=osim_muscls_dict['patella_r'].index, columns=osim_muscls_dict['patella_r'].columns)

patella_r_transformed_bone_df.to_csv(os.path.join(control_path, 'patella_r_transformed_bone.csv'))
patella_r_transformed_muscle_df.to_csv(os.path.join(control_path, 'patella_r_transformed_muscle.csv'))

patella_r_transformed.save(os.path.join(control_path, f'patella_r_transformed.stl'))

In [72]:
patella_bone_markers = list(mri_dict['patella_r'].index)

patella_rotate = femur_r_axes.transform_patella(patella_r_transformed_bone, 
                                                  patella_bone_markers,
                                                  patella_muscles_numpy = patella_r_transformed_muscle,  
                                                  patella_surface_numpy = patella_r_transformed.points)

patella_r_bone_in_child = patella_rotate[0]
patella_r_muscle_in_child = patella_rotate[1] 
patella_r_surface_in_child = pv.PolyData(patella_rotate[2], patella_r_transformed.faces)

In [73]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_r_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(patella_r_muscle_in_child),  color='black', point_size=5.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f162f30c10_13&reconnect=auto" class="pyvi…

# femur_l  tps

In [74]:
mri_dict['femur_l'].index

Index(['knee_l_med', 'knee_l_lat', 'femur_l_center', 'patella_l_in_femur_l',
       'knee_l_center_in_femur_l', 'gr_troch_as_l', 'gr_troch_lat_l',
       'femur_l_midshaft_anter', 'femur_l_midshaft_poster', 'ls_troch_l',
       'femur_l_poster_diaph_25', 'femur_l_poster_diaph_75',
       'femur_l_anter_diaph_25', 'gr_troch_ps_l', 'femur_l_anter_diaph_75'],
      dtype='object', name='names')

In [75]:
# exclude markers to control for the quality of the spline
to_exclude_femur_l = [] #, 'gr_troch_ps_l','gr_troch_lat_l', 'gr_troch_as_l'

# calculate the spline
femur_l_tps = OneBodyTPS('femur_l', osim_dict['femur_l'], 
                         mri_dict['femur_l'], 
                         osim_skin = osim_skin_markers_dict['femur_l'],
                         osim_muscle = osim_muscls_dict['femur_l'], 
                         exclude_bone_markers = to_exclude_femur_l)

# transform all objects
femur_l_osim_bone = osim_dict['femur_l']
femur_l_transformed_bone = femur_l_tps.transformed_bone
femur_l_transformed_skin = femur_l_tps.transformed_skin
femur_l_transformed_muscle = femur_l_tps.transformed_muscle
femur_l_transformed_wraps = femur_l_tps.apply_tps_to_wraps(osim_wraps_dict['femur_l'])

l_femur = ImportVTP(vtp_path, 'l_femur.stl')
femur_l_transformed = femur_l_tps.apply_spline_to_surface(l_femur.mesh)

# plot results
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_transformed_bone),  color='blue', point_size=10.0)
if len(to_exclude_femur_l) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['femur_l'].loc[to_exclude_femur_l].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f0f0b7f050_14&reconnect=auto" class="pyvi…

In [76]:
len(osim_skin_markers_dict['femur_r'])

8

In [78]:
# # record results into the control folder
femur_l_transformed_bone_df = pd.DataFrame(femur_l_transformed_bone, index=femur_l_osim_bone.index, columns=femur_l_osim_bone.columns)
femur_l_transformed_skin_df = pd.DataFrame(femur_l_transformed_skin, index=osim_skin_markers_dict['femur_l'].index, columns=osim_skin_markers_dict['femur_l'].columns)

femur_l_transformed_muscle_df = pd.DataFrame(femur_l_transformed_muscle, index=osim_muscls_dict['femur_l'].index, columns=osim_muscls_dict['femur_l'].columns)
femur_l_transformed_wraps_translation_df = pd.DataFrame(femur_l_transformed_wraps[2], index=osim_wraps_dict['femur_l']['name'], columns=['r', 'a', 's'])

femur_l_transformed_bone_df.to_csv(os.path.join(control_path, 'femur_l_transformed_bone.csv'))
femur_l_transformed_skin_df.to_csv(os.path.join(control_path, 'femur_l_transformed_skin.csv'))

femur_l_transformed_muscle_df.to_csv(os.path.join(control_path, 'femur_l_transformed_muscle.csv'))
femur_l_transformed_wraps_translation_df.to_csv(os.path.join(control_path, 'femur_l_transformed_wraps_translation.csv'))

femur_l_transformed.save(os.path.join(control_path, f'femur_l_transformed.stl'))

In [79]:
# create rotations to child
bone_markers = list(mri_dict['femur_l'].index)
femur_l_axes = GetFemurAxes(femur_l_transformed_bone,
                            bone_markers, 
                            femur_skin_numpy = femur_l_transformed_skin,
                            femur_muscles_numpy = femur_l_transformed_muscle,
                            femur_wraps_numpy = femur_l_transformed_wraps[2],
                            femur_surface_numpy = femur_l_transformed.points)
# rotate to child
femur_l_bone_in_child = femur_l_axes.bone_transformed
femur_l_skin_in_child = femur_l_axes.femur_skin
femur_l_muscles_in_child = femur_l_axes.femur_muscles
femur_l_wrap_translations_in_child = femur_l_axes.femur_wraps
femur_l_surface_in_child = pv.PolyData(femur_l_axes.femur_surface, femur_l_transformed.faces)

In [80]:
# plot results of the rotation
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_femur.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['femur_l'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['femur_l']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(femur_l_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(femur_l_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(femur_l_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f186b5fc10_15&reconnect=auto" class="pyvi…

# patella_l  tps

In [81]:
patella_l_exclude =  None

patella_l_tps = OneBodyTPS('patella_l', osim_dict['patella_l'], 
                         mri_dict['patella_l'], 
                         osim_muscle = osim_muscls_dict['patella_l'], 
                         osim_skin = None,
                         exclude_bone_markers = patella_l_exclude)

patella_l_osim_bone = osim_dict['patella_l']
patella_l_transformed_bone = patella_l_tps.transformed_bone
patella_l_transformed_muscle = patella_l_tps.transformed_muscle

l_patella = ImportVTP(vtp_path, 'l_patella.vtp')

patella_l_transformed = patella_l_tps.apply_spline_to_surface(l_patella.mesh)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_transformed_bone),  color='blue', point_size=10.0)
# if len(patella_r_exclude) >= 1:
#actor = plotter.add_mesh(pv.PolyData(osim_mri_bone_by_bodies.patella_r_mri.loc[patella_r_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f19dcfda50_16&reconnect=auto" class="pyvi…

In [82]:
# record results into the control folder
patella_l_transformed_bone_df = pd.DataFrame(patella_l_transformed_bone, index=patella_l_osim_bone.index, columns=patella_l_osim_bone.columns)
patella_l_transformed_muscle_df = pd.DataFrame(patella_l_transformed_muscle, index=osim_muscls_dict['patella_l'].index, columns=osim_muscls_dict['patella_l'].columns)

patella_l_transformed_bone_df.to_csv(os.path.join(control_path, 'patella_l_transformed_bone.csv'))
patella_l_transformed_muscle_df.to_csv(os.path.join(control_path, 'patella_l_transformed_muscle.csv'))

patella_l_transformed.save(os.path.join(control_path, f'patella_l_transformed.stl'))

In [83]:
patella_bone_markers = list(mri_dict['patella_l'].index)

patella_rotate = femur_l_axes.transform_patella(patella_l_transformed_bone, 
                                                  patella_bone_markers,
                                                  patella_muscles_numpy = patella_l_transformed_muscle,  
                                                  patella_surface_numpy = patella_l_transformed.points)

patella_l_bone_in_child = patella_rotate[0]
patella_l_muscle_in_child = patella_rotate[1] 
patella_l_surface_in_child = pv.PolyData(patella_rotate[2], patella_l_transformed.faces)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_patella.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(patella_l_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(patella_l_muscle_in_child),  color='black', point_size=5.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f19dc67b10_17&reconnect=auto" class="pyvi…

# tibia_r tps

In [84]:
tibia_r_exclude =  [] #'tibia_r_anter_diaph_75',

tibia_r_tps = OneBodyTPS('tibia_r', osim_dict['tibia_r'], 
                         mri_dict['tibia_r'], 
                         osim_skin = osim_skin_markers_dict['tibia_r'],
                         osim_muscle = osim_muscls_dict['tibia_r'], 
                         exclude_bone_markers = tibia_r_exclude)

tibia_r_osim_bone = osim_dict['tibia_r']
tibia_r_transformed_bone = tibia_r_tps.transformed_bone
tibia_r_transformed_skin = tibia_r_tps.transformed_skin
tibia_r_transformed_muscle = tibia_r_tps.transformed_muscle
tibia_r_transformed_wraps = tibia_r_tps.apply_tps_to_wraps(osim_wraps_dict['tibia_r'])

r_tibia = ImportVTP(vtp_path, 'r_tibia.stl')
r_fibula = ImportVTP(vtp_path, 'r_fibula.stl')

tibia_r_transformed = tibia_r_tps.apply_spline_to_surface(r_tibia.mesh)
fibula_r_transformed = tibia_r_tps.apply_spline_to_surface(r_fibula.mesh)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_r_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_transformed_bone),  color='blue', point_size=10.0)
if len(tibia_r_exclude) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['tibia_r'].loc[tibia_r_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f199ecdc90_18&reconnect=auto" class="pyvi…

In [85]:
# # record results into the control folder
tibia_r_transformed_bone_df = pd.DataFrame(tibia_r_transformed_bone, index=tibia_r_osim_bone.index, columns=tibia_r_osim_bone.columns)
tibia_r_transformed_skin_df = pd.DataFrame(tibia_r_transformed_skin, index=osim_skin_markers_dict['tibia_r'].index, columns=osim_skin_markers_dict['tibia_r'].columns)
tibia_r_transformed_muscle_df = pd.DataFrame(tibia_r_transformed_muscle, index=osim_muscls_dict['tibia_r'].index, columns=osim_muscls_dict['tibia_r'].columns)
tibia_r_transformed_wraps_translation_df = pd.DataFrame(tibia_r_transformed_wraps[2], index=osim_wraps_dict['tibia_r']['name'], columns=['r', 'a', 's'])

tibia_r_transformed_bone_df.to_csv(os.path.join(control_path, 'tibia_r_transformed_bone.csv'))
tibia_r_transformed_skin_df.to_csv(os.path.join(control_path, 'tibia_r_transformed_skin.csv'))
tibia_r_transformed_muscle_df.to_csv(os.path.join(control_path, 'tibia_r_transformed_muscle.csv'))
tibia_r_transformed_wraps_translation_df.to_csv(os.path.join(control_path, 'tibia_r_transformed_wraps_translation.csv'))

tibia_r_transformed.save(os.path.join(control_path, f'tibia_r_transformed.stl'))
fibula_r_transformed.save(os.path.join(control_path, f'fibula_r_transformed.stl'))

In [86]:
# create a rotation to child
bone_markers = list(mri_dict['tibia_r'].index)

tibia_r_axes = GetTibiaAxes(bone_markers = bone_markers, bone_numpy = tibia_r_transformed_bone)

# apply the rotaion
tibia_r_bone_in_child = tibia_r_axes.bone_transformed
tibia_r_skin_in_child = tibia_r_axes.apply_to_not_bone(tibia_r_transformed_skin)
tibia_r_muscles_in_child = tibia_r_axes.apply_to_not_bone(tibia_r_transformed_muscle)
tibia_r_wrap_translations_in_child = tibia_r_axes.apply_to_not_bone(tibia_r_transformed_wraps[2])

tibia_r_surface_in_child = pv.PolyData(
    tibia_r_axes.apply_to_not_bone(tibia_r_transformed.points), 
    tibia_r_transformed.faces)

fibula_r_surface_in_child = pv.PolyData(
    tibia_r_axes.apply_to_not_bone(fibula_r_transformed.points), 
    fibula_r_transformed.faces)

the determinant is less than zero, recalculate r


In [87]:
# plot results of the rotation
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(r_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(r_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['tibia_r'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['tibia_r']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_r_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_r_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(tibia_r_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(tibia_r_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f1c7ab7190_19&reconnect=auto" class="pyvi…

# tibia_l  tps

In [88]:
tibia_l_exclude = []

tibia_l_tps = OneBodyTPS('tibia_l', osim_dict['tibia_l'], 
                         mri_dict['tibia_l'], 
                         osim_muscle = osim_muscls_dict['tibia_l'], 
                         osim_skin = osim_skin_markers_dict['tibia_l'],
                         exclude_bone_markers = tibia_l_exclude)

tibia_l_osim_bone = osim_dict['tibia_l']
tibia_l_transformed_bone = tibia_l_tps.transformed_bone
tibia_l_transformed_muscle = tibia_l_tps.transformed_muscle
tibia_l_transformed_skin = tibia_l_tps.transformed_skin
tibia_l_transformed_wraps = tibia_l_tps.apply_tps_to_wraps(osim_wraps_dict['tibia_l'])

l_tibia = ImportVTP(vtp_path, 'l_tibia.stl')
l_fibula = ImportVTP(vtp_path, 'l_fibula.stl')

tibia_l_transformed = tibia_l_tps.apply_spline_to_surface(l_tibia.mesh)
fibula_l_transformed = tibia_l_tps.apply_spline_to_surface(l_fibula.mesh)

plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(l_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_osim_bone.to_numpy()),  color='blue', point_size=10.0) 
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_l_transformed, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_transformed_bone),  color='blue', point_size=10.0)
if len(tibia_l_exclude) >= 1:
    actor = plotter.add_mesh(pv.PolyData(mri_dict['tibia_l'].loc[tibia_l_exclude].to_numpy()), color='green', point_size=10.0)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f201c58ed0_20&reconnect=auto" class="pyvi…

In [89]:
# #record results into the control folder
tibia_l_transformed_bone_df = pd.DataFrame(tibia_l_transformed_bone, index=tibia_l_osim_bone.index, columns=tibia_l_osim_bone.columns)
tibia_l_transformed_skin_df = pd.DataFrame(tibia_l_transformed_skin, index=osim_skin_markers_dict['tibia_l'].index, columns=osim_skin_markers_dict['tibia_l'].columns)
tibia_l_transformed_muscle_df = pd.DataFrame(tibia_l_transformed_muscle, index=osim_muscls_dict['tibia_l'].index, columns=osim_muscls_dict['tibia_l'].columns)
tibia_l_transformed_wraps_translation_df = pd.DataFrame(tibia_l_transformed_wraps[2], index=osim_wraps_dict['tibia_l']['name'], columns=['r', 'a', 's'])

tibia_l_transformed_bone_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_bone.csv'))
tibia_l_transformed_skin_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_skin.csv'))
tibia_l_transformed_muscle_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_muscle.csv'))
tibia_l_transformed_wraps_translation_df.to_csv(os.path.join(control_path, 'tibia_l_transformed_wraps_translation.csv'))

tibia_l_transformed.save(os.path.join(control_path, f'tibia_l_transformed.stl'))
fibula_l_transformed.save(os.path.join(control_path, f'fibula_l_transformed.stl'))

In [90]:
bone_markers = list(mri_dict['tibia_l'].index)

tibia_l_axes = GetTibiaAxes(bone_markers = bone_markers, bone_numpy = tibia_l_transformed_bone)
tibia_l_bone_in_child = tibia_l_axes.bone_transformed
tibia_l_skin_in_child = tibia_l_axes.apply_to_not_bone(tibia_l_transformed_skin)
tibia_l_muscles_in_child = tibia_l_axes.apply_to_not_bone(tibia_l_transformed_muscle)
tibia_l_wrap_translations_in_child = tibia_l_axes.apply_to_not_bone(tibia_l_transformed_wraps[2])

tibia_l_surface_in_child = pv.PolyData(
    tibia_l_axes.apply_to_not_bone(tibia_l_transformed.points), 
    tibia_l_transformed.faces)

fibula_l_surface_in_child = pv.PolyData(
    tibia_l_axes.apply_to_not_bone(fibula_l_transformed.points), 
    fibula_l_transformed.faces)

In [91]:
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_tibia.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(l_fibula.mesh, color='lightblue', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_osim_bone.to_numpy()),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(osim_muscls_dict['tibia_l'].to_numpy()),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(np.stack(osim_wraps_dict['tibia_l']['translation'].to_numpy())),  color='red', point_size=10.0)  
plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(fibula_l_surface_in_child, color='red', opacity=0.5, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tibia_l_bone_in_child),  color='blue', point_size=10.0)
actor = plotter.add_mesh(pv.PolyData(tibia_l_muscles_in_child),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(tibia_l_wrap_translations_in_child),  color='red', point_size=10.0)

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60660/index.html?ui=P_0x2f201cfa750_21&reconnect=auto" class="pyvi…